In [1]:
import pandas as pd
import useful_functions as use
import psycopg2
import pyodbc
from datetime import datetime, date

In [2]:
conn = psycopg2.connect(
    host='oneview-prd-content-aurora-cluster.cluster-cxgp9osuwqi3.us-east-1.rds.amazonaws.com',
    database='oneview_content',
    user='oneviewadmin',
    password='VFauC8cRXqKapC8A')

In [3]:
username = 'vigrose'
password_edw = 'Slytherin~10946'
w = "DSN=PRDDW; UID={}; PWD={}".format(username, password_edw)
AMAEDW = pyodbc.connect(w)

In [4]:
password_aims = 'DQA8K@T7pXEqEt'
s = "DSN=aims_prod; UID={}; PWD={}".format(username, password_aims)
informix = pyodbc.connect(s)

In [5]:
ov_me_sql = '''
    SELECT medical_education_number,type FROM ONEVIEW.physician
    '''
ov_me = pd.read_sql_query(ov_me_sql, conn)

In [6]:
QUERY = \
        """
        SELECT DISTINCT 
        M.ME, E.ENTITY_ID 
        FROM 
        AMAEDW.PARTY_ID_TO_ME_VW M, AMAEDW.PARTY_ENTITY_VW E 
        WHERE 
        M.PARTY_ID=E.PARTY_ID
        """
w = "DSN=PRDDW; UID={}; PWD={}".format(username, password_edw)
AMAEDW = pyodbc.connect(w)
ENTITY_TO_ME = pd.read_sql(con=AMAEDW, sql=QUERY)

In [8]:
query_3 = \
    f'''
    SELECT
    ENTITY_ID,
    PERSON_TYPE
    FROM
    informix.PERSON_ROLE_ET
    WHERE
    ACTIVE_IND = 'Y'
    '''
aims_type = pd.read_sql(con=informix, sql=query_3)
aims_type.head()

,entity_id,person_type
0,1519228,S
1,1525234,S
2,1351372,S
3,1352542,S
4,1359445,S


In [14]:
ENTITY_TO_ME['entity_id'] = [int(x) for x in ENTITY_TO_ME.ENTITY_ID]
ALL = pd.merge(ov_me, ENTITY_TO_ME, left_on='medical_education_number', right_on='ME', how='left')
ALL = pd.merge(ALL, aims_type, on='entity_id', how='left')

In [17]:
ALL['ONEVIEW_TYPE'] = [x[0] for x in ALL.type]

In [20]:
ALL[ALL.ONEVIEW_TYPE!=ALL.person_type]

,medical_education_number,type,ME,ENTITY_ID,entity_id,person_type,ONEVIEW_TYPE
27237,00401861556,Physician,00401861556,1438940,1438940,R,P
83890,00519200791,Physician,00519200791,48270296,48270296,R,P
96997,00702041139,Physician,00702041139,21819898,21819898,R,P
308504,02002081272,Physician,02002081272,28341010,28341010,R,P
438066,02576142296,Resident,02576142296,60563711,60563711,P,R
625483,03545160730,Physician,03545160730,45506288,45506288,R,P
872414,04720040561,Physician,04720040561,21817324,21817324,R,P
877992,04802132200,Physician,04802132200,42887425,42887425,R,P
895039,04812050287,Physician,04812050287,22229658,22229658,R,P
931329,04878130052,Physician,04878130052,42872579,42872579,R,P
